In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import pandas as pd

In [2]:
#Get labs urls
link = uReq('https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Laboratory&CycleBeginYear=2013')
page_soup = soup(link.read(), "html.parser")
containers = page_soup.findAll("table", {"class":"table table-bordered table-header-light table-striped table-hover table-hover-light nein-scroll"})
table = containers[0].findAll('a')
ls_labs = []

for i in range(0,len(table)):
    ls_labs.append('https://wwwn.cdc.gov'+table[i].get('href'))
ls_labs = list(filter(lambda x: 'htm' in x,ls_labs))

In [3]:
#Get questionnaire urls
link = uReq('https://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Questionnaire&CycleBeginYear=2013')
page_soup = soup(link.read(), "html.parser")
containers = page_soup.findAll("table", {"class":"table table-bordered table-header-light table-striped table-hover table-hover-light nein-scroll"})
table = containers[0].findAll('a')
ls_questionnaire = []

for i in range(0,len(table)):
    ls_questionnaire.append('https://wwwn.cdc.gov'+table[i].get('href'))
ls_questionnaire = list(filter(lambda x: 'htm' in x,ls_questionnaire))

In [4]:
#All urls

list_url = {'Demographic':['https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.htm'],
            'Examinations':['https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BPX_H.htm','https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BMX_H.htm','https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.htm','https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/CSX_H.htm'],
            'Labs':ls_labs,
            'Diet':['https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DR1TOT_H.htm'],
            'Questionnaire':ls_questionnaire}

In [11]:
def scraper(list_url):
    for dataset in list_url:
        ls_final = []
        for n in range(0,len(list_url[dataset])):
            page_url = list_url[dataset][n]
            # opens the connection and downloads html page from url
            link = uReq(page_url)
            # parses html into a soup data structure to traverse html as if it were a json data type.
            page_soup = soup(link.read(), "html.parser")
            #link.close()
            # get all tables
            containers = page_soup.findAll("div", {"class": "pagebreak"})
            df = pd.DataFrame()
            #get all rows
            for i in range(1,len(containers)):
                # title
                title = containers[i].h3.get_text()
                short_title = containers[i].find("dd",{"class":"info"}).get_text()
                # body values
                try:
                    table = containers[i].tbody.findAll('tr')
                    ls = []
                    #ls = ls.append(title)
                    for i in table:
                        ls.append(i.text)
                    #clean the \n and append to the table
                    for i in ls:
                        ls_final.append(short_title + "|" + title + i.replace('\n','|'))
                except:
                    break
            print(page_url)
        #conver to dataframe
        df = pd.DataFrame(ls_final)
        #expand
        cleaned = df[0].str.split('|', expand=True)
        #drop other columns
        cleaned = cleaned.iloc[:,:5]
        #rename columns
        cleaned.columns = ['short_title','title','code or value','description','count']

        #save to csv
        cleaned.to_csv(r'{title}.csv'.format(title = dataset),index=False)

In [12]:
scraper(list_url)

https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BPX_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BMX_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/CSX_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/ALB_CR_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/ALD_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/ALDS_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/SSTOCA_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/APOB_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/UTAS_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/UTASS_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/UAS_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/UASS_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/PBCD_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/IHGEM_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BFRPOL_H.htm
https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/CHLMDA_H.htm
https://wwwn.cdc.gov/